In [1]:
import requests  # send request
from bs4 import BeautifulSoup  # parse web pages
import pandas as pd  # csv
from time import sleep  # wait
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
# import json
from tqdm import tqdm  # progress bar

In [2]:
# header100 = {
#     "authority": "arxiv.org",
#     "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7",
#     "accept-language": "zh-CN,zh;q=0.9,en;q=0.8,zh-HK;q=0.7,en-GB;q=0.6,en-US;q=0.5,zh-TW;q=0.4",
#     "cache-control": "max-age=0",
#     "if-modified-since": "Mon, 04 Mar 2024 04:25:40 GMT",
#     "if-none-match": '"Mon, 04 Mar 2024 04:25:40 GMT"',
#     # "referer": " https://curlconverter.com/",
#     "sec-ch-ua": '"Chromium";v="122", "Not(A:Brand";v="24", "Microsoft Edge";v="122"',
#     "sec-ch-ua-mobile": "?0",
#     "sec-ch-ua-platform": '"Windows"',
#     "sec-fetch-dest": "document",
#     "sec-fetch-mode": "navigate",
#     "sec-fetch-site": "none",
#     "sec-fetch-user": "?1",
#     "upgrade-insecure-requests": "1",
#     "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/122.0.0.0 Safari/537.36 Edg/122.0.0.0",
# }

In [3]:
url = "https://arxiv.org/list/cs/pastweek?skip=0&show=100"
page = requests.get(url)
soup100 = BeautifulSoup(page.content, "html.parser")

# Collecting the title, Subjects, Authors, and abstract

In [4]:
all100 = soup100.find("dl")

title_list = all100.find_all("div", class_="list-title mathjax")
title_text = title_list[0].get_text(strip=True)
title_text = title_text[title_text.find("Title:")+6:]
title_text

'Point Could Mamba: Point Cloud Learning via State Space Model'

In [5]:
subject_list = all100.find_all("div", class_="list-subjects")

subject_text = subject_list[0].find("span", class_="primary-subject").get_text(strip=True)
subject_text

'Computer Vision and Pattern Recognition (cs.CV)'

In [6]:
author_list= all100.find_all("div", class_="list-authors")

author_text = author_list[0].get_text(strip=True)
author_text = author_text[author_text.find("Authors:")+8:]
author_text

'Tao Zhang,Xiangtai Li,Haobo Yuan,Shunping Ji,Shuicheng Yan'

In [7]:
paper_id_list = all100.find_all("a", title="Abstract")

paper_id = paper_id_list[0].get_text(strip=True)
paper_id = paper_id[paper_id.find("arXiv:")+6:]
url_abstract = "https://arxiv.org/abs/"+ str(paper_id)
page_abstract = requests.get(url_abstract)
soup_abstract = BeautifulSoup(page_abstract.content, "html.parser")
soup_abstract.find("blockquote", class_="abstract mathjax").get_text(strip=True)[9:]

"In this work, for the first time, we demonstrate that Mamba-based point cloud methods can outperform point-based methods. Mamba exhibits strong global modeling capabilities and linear computational complexity, making it highly attractive for point cloud analysis. To enable more effective processing of 3-D point cloud data by Mamba, we propose a novel Consistent Traverse Serialization to convert point clouds into 1-D point sequences while ensuring that neighboring points in the sequence are also spatially adjacent. Consistent Traverse Serialization yields six variants by permuting the order of x, y, and z coordinates, and the synergistic use of these variants aids Mamba in comprehensively observing point cloud data. Furthermore, to assist Mamba in handling point sequences with different orders more effectively, we introduce point prompts to inform Mamba of the sequence's arrangement rules. Finally, we propose positional encoding based on spatial coordinate mapping to inject positional 

# save into `.csv` file

In [8]:
df = pd.DataFrame(columns=["Title", "Subject", "Authors", "Abstract"])

In [9]:
for i in tqdm(range(100)):
    try:
        title_text = title_list[i].get_text(strip=True)
        title_text = title_text[title_text.find("Title:")+6:]
        subject_text = subject_list[i].find("span", class_="primary-subject").get_text(strip=True)
        author_text = author_list[i].get_text(strip=True)
        author_text = author_text[author_text.find("Authors:")+8:]
        paper_id = paper_id_list[i].get_text(strip=True)
        paper_id = paper_id[paper_id.find("arXiv:")+6:]
        url_abstract = "https://arxiv.org/abs/"+ str(paper_id)

        ################################################################################
        for _ in range(10):  # 尝试 多次
            try:
                page_abstract = requests.get(url_abstract, timeout=5)
                break
            except requests.exceptions.RequestException:
                sleep(10)
        else:
            print("skip the paper with index = ", i)
            continue  # 如果 几 次尝试都失败，跳过这个循环的剩余部分并开始下一个循环
        ################################################################################

        page_abstract = requests.get(url_abstract)
        soup_abstract = BeautifulSoup(page_abstract.content, "html.parser")
        abstract_text = soup_abstract.find("blockquote", class_="abstract mathjax").get_text(strip=True)[9:]
        df = df._append({"Title": title_text, "Subject": subject_text, "Authors": author_text, "Abstract": abstract_text}, ignore_index=True)
        sleep(2)
    except Exception as e:
        print(f"An error occurred: {e}")

  9%|▉         | 9/100 [00:32<05:31,  3.64s/it]

An error occurred: HTTPSConnectionPool(host='arxiv.org', port=443): Max retries exceeded with url: /abs/2403.00725 (Caused by ProxyError('Cannot connect to proxy.', ConnectionResetError(10054, '远程主机强迫关闭了一个现有的连接。', None, 10054, None)))


100%|██████████| 100/100 [06:55<00:00,  4.15s/it]


In [10]:
df.to_csv("arxiv.csv", index=False)
df

,Title,Subject,Authors,Abstract
0,Point Could Mamba: Point Cloud Learning via St...,Computer Vision and Pattern Recognition (cs.CV),"Tao Zhang,Xiangtai Li,Haobo Yuan,Shunping Ji,S...","In this work, for the first time, we demonstra..."
1,Mitigating Reversal Curse via Semantic-aware P...,Computation and Language (cs.CL),"Qingyan Guo,Rui Wang,Junliang Guo,Xu Tan,Jiang...",While large language models (LLMs) have achiev...
2,An Experimental Study of Low-Latency Video Str...,Multimedia (cs.MM),"Imran Khan,Tuyen X. Tran,Matti Hiltunen,Theodo...",Low-latency video streaming over 5G has become...
3,AtP*: An efficient and scalable method for loc...,Machine Learning (cs.LG),"János Kramár,Tom Lieberum,Rohin Shah,Neel Nand...",Activation Patching is a method of directly co...
4,Neural Acceleration of Incomplete Cholesky Pre...,"Distributed, Parallel, and Cluster Computing (...","Joshua Dennis Booth,Hongyang Sun,Trevor Garnett",The solution of a sparse system of linear equa...
...,...,...,...,...
94,Do Zombies Understand? A Choose-Your-Own-Adven...,Computation and Language (cs.CL),"Ariel Goldstein,Gabriel Stanovsky",Recent advances in LLMs have sparked a debate ...
95,"Graph Homomorphism, Monotone Classes and Bound...",Computational Complexity (cs.CC),"Tala Eagling-Vose,Barnaby Martin,Daniel Paulus...",A recent paper describes a framework for study...
96,Analyzing Divergence for Nondeterministic Prob...,Logic in Computer Science (cs.LO),"Hao Wu,Yuxi Fu,Huan Long,Xian Xu,Wenbo Zhang",Branching and weak probabilistic bisimilaritie...
97,Multiple Ways of Working with Users to Develop...,Human-Computer Interaction (cs.HC),"Amal Nanavati,Max Pascher,Vinitha Ranganeni,Et...",Despite the growth of physically assistive rob...
